In [2]:
from batchRequesthandler import BatchRequestHandler

handler = BatchRequestHandler()

# Post Batch
batch_id = handler.post_batch()
print("Batch ID:", batch_id)

Attempting Mathpix POST request
Request Complete
Batch ID: 21278625


In [3]:
import json

# Get Batch Results
results_json = handler.get_batch_results(batch_id)

Attempting Mathpix GET request for batch 21278625
Request Complete


In [11]:
# Process the results in  a convenient way
image_ids = results_json["keys"]
results_data = results_json["results"]

output_json = []
errors_json = []
success_count, error_count = 0, 0
for image_id in image_ids:
    if "error" in results_data[image_id].keys():
        # Track images where errors occured and continue
        errors_json.append(
            {
                "id": image_id,
                "error": results_data[image_id]["error_info"],
                "link": f"https://raw.githubusercontent.com/imzoc/mathpix-annotation/master/mathml-images/images_filtered/{image_id}.png"
            }
        )
        error_count += 1
        continue
    else:
        current_image_result_data = results_data[image_id]["data"]

        mathml_res = []
        latex_res = []
        for row in current_image_result_data:
            if row["type"] == "mathml":
                mathml_res.append(row["value"])
            else:
                latex_res.append(row["value"])

        current_image_consolidated_result_data = {
            "id": image_id,
            "mathml": " ".join(mathml_res),
            "latex": " ".join(latex_res),
        }
        output_json.append(current_image_consolidated_result_data)
        success_count += 1


# Save results
with open("../latex_data_dev.js", "w") as f:
    f.write(f"eqn_src = {json.dumps(output_json, indent=4)}")
    print(
        f"{success_count} images were successfully processed. The results have been saved to {f.name}"
    )

with open("mathpix_errors.json", "w") as f:
    json.dump(errors_json, f, indent=4)
    print(
        f"{error_count} images encountered errors. The results have been saved to {f.name}"
    )

3994 images were successfully processed. The results have been saved to ../latex_data_dev.js
78 images encountered errors. The results have been saved to mathpix_errors.json
